In [ ]:
pip install -U tensorflow_decision_forests

In [ ]:
import numpy as np  
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
tf.random.set_seed(123)

from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score,roc_curve

In [ ]:
dataset_cancer = datasets.load_breast_cancer()    # 加载癌细胞数据集

#print(dataset_cancer['DESCR'])

df = pd.DataFrame(dataset_cancer.data, columns=dataset_cancer.feature_names)  

df['label'] = dataset_cancer.target

print(df.shape)

df.head()

In [ ]:

# holdout验证法： 按3：7划分测试集 训练集
x_train, x_test= train_test_split(df, test_size=0.3)

# EDA分析：数据统计指标
x_train.describe(include='all')

In [ ]:
import tensorflow_decision_forests as tfdf


target = "label"

train_ds = tfdf.keras.pd_dataframe_to_tf_dataset(x_train, label=target, task=tfdf.keras.Task.CLASSIFICATION)
test_ds = tfdf.keras.pd_dataframe_to_tf_dataset(x_test, label=target, task=tfdf.keras.Task.CLASSIFICATION)


In [ ]:
# 模型参数
model_tf = tfdf.keras.GradientBoostedTreesModel(loss="BINARY_FOCAL_LOSS")

# 模型训练
model_tf.compile(metrics=["acc"])
model_tf.fit(x=train_ds,validation_freq=0.1)

In [ ]:
## 模型评估
evaluation = model_tf.evaluate(test_ds,return_dict=True)
probs = model_tf.predict(test_ds)
fpr, tpr, _ = roc_curve(x_test.label, probs)
plt.plot(fpr, tpr)
plt.title('ROC curve')
plt.xlabel('false positive rate')
plt.ylabel('true positive rate')
plt.xlim(0,)
plt.ylim(0,)
plt.show()
print(evaluation)

In [ ]:
# 还可以输出每个样本所在的叶子节点，方便GBDT+DNN的融合
model_tf.predict_get_leaves(test_ds)

In [ ]:
print(model_tf.summary())

In [ ]:
tfdf.model_plotter.plot_model_in_colab(model_tf, tree_idx=0, max_depth=10)